#### **Author: Wissem Khlifi**

# 01 - BigQuery - Table Data Source
Use BigQuery to load and prepare data for machine learning. This includes understanding the data source,
feature selection, and splitting datasets into testing, validation, and training datasets.



## Source Data
Specify the BigQuery table that will be used as the data source.
This table contains the dataset for the machine learning project.
BigQuery Table: `<Your Project ID>.ml_datasets.ulb_fraud_detection`

### Explanation of Data Splitting

Splitting data into training, validation, and testing sets is crucial in machine learning to ensure that
the model generalizes well to new, unseen data. It helps in evaluating the model's performance and tuning it effectively.

- Training Data: This is the portion of the dataset used to train the model. The model learns patterns and relationships
in the training data to make predictions. It typically comprises the largest portion of the dataset.

- Validation Data: This subset is used to tune the model's hyperparameters and to provide an unbiased evaluation
of the model during training. It helps in selecting the best model architecture and avoiding overfitting.

- Testing Data: The test set is used to evaluate the final model's performance. It provides an unbiased measure
of the model's accuracy and generalization to new data, as it is not used during the training or validation phases.



## Setup

### inputs

Get the GCP project ID from the gcloud configuration
This ensures the notebook uses the correct GCP project for all operations.


In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'bootkon-test24mun-8303'

### Define variables for the rest of the Notebook


In [2]:
# Specifying the region is necessary for certain GCP services to function correctly.
REGION = 'us-central1'
# Define experiment series : This helps in organizing and tracking different experiment runs.
EXPERIMENT = '01'
SERIES = '01'

# Define source data parameters
# These variables specify the BigQuery dataset and table to be used.
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'ml_datasets'
BQ_TABLE = 'ulb_fraud_detection'

# Construct the full BigQuery source path
# This combines project ID, dataset, and table into a single reference for BigQuery operations

BQ_SOURCE = f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}'
BQ_SOURCE


'bootkon-test24mun-8303.ml_datasets.ulb_fraud_detection'

### Import necessary Packages:
These imports are required for interacting with BigQuery, Google Cloud Storage, and Google Cloud Authentication.


In [3]:
from google.cloud import bigquery
from google.cloud import storage
from google.auth import compute_engine, default
import google.auth
import google.auth.transport.requests

### Clients
Initialize clients for GCP services
Creating client instances allows interaction with GCP services within the notebook.


In [4]:
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)

### Parameters
Set parameters
Using the project ID as the bucket name for simplicity and to ensure uniqueness.


In [5]:
BUCKET = PROJECT_ID

### Retrieve and Review a Sample From The Table:
This section demonstrates how to query a sample of data from the BigQuery table to review it before further processing.

> **Note:** The `LIMIT 5` statement does limit the number of rows returned by BigQuery to 5 but BigQuery still does a full table scan.  
If you have a table larger than 1GB and want to limit the rows scanned for a quick review like then then replacing `LIMIT 5` with `TABLESAMPLE SYSTEM (1 PERCENT)` would be more efficient.  For tables under 1GB it will still return the full table.  
More on [Table Sampling](https://cloud.google.com/bigquery/docs/table-sampling)

In [6]:
# Construct a query to retrieve a sample of data from the BigQuery table
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
#LIMIT 5
"""
# Execute the query and convert the result to a pandas dataframe for easy manipulation and review in Python
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,282.0,-0.356466,0.725418,1.971749,0.831343,0.369681,-0.107776,0.751610,-0.120166,-0.420675,...,0.020804,0.424312,-0.015989,0.466754,-0.809962,0.657334,-0.043150,-0.046401,0.00,0
1,14332.0,1.071950,0.340678,1.784068,2.846396,-0.751538,0.403028,-0.734920,0.205807,1.092726,...,-0.169632,-0.113604,0.067643,0.468669,0.223541,-0.112355,0.014015,0.021504,0.00,0
2,32799.0,1.153477,-0.047859,1.358363,1.480620,-1.222598,-0.481690,-0.654461,0.128115,0.907095,...,0.125514,0.480049,-0.025964,0.701843,0.417245,-0.257691,0.060115,0.035332,0.00,0
3,35799.0,-0.769798,0.622325,0.242491,-0.586652,0.527819,-0.104512,0.209909,0.669861,-0.304509,...,0.152738,0.255654,-0.130237,-0.660934,-0.493374,0.331855,-0.011101,0.049089,0.00,0
4,36419.0,1.047960,0.145048,1.624573,2.932652,-0.726574,0.690451,-0.627288,0.278709,0.318434,...,0.078499,0.658942,-0.067810,0.476882,0.526830,0.219902,0.070627,0.028488,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142066,154599.0,0.667714,3.041502,-5.845112,5.967587,0.213863,-1.462923,-2.688761,0.677764,-3.447596,...,0.329760,-0.941383,-0.006075,-0.958925,0.239298,-0.067356,0.821048,0.426175,6.74,1
142067,90676.0,-2.405580,3.738235,-2.317843,1.367442,0.394001,1.919938,-3.106942,-10.764403,3.353525,...,10.005998,-2.454964,1.684957,0.118263,-1.531380,-0.695308,-0.152502,-0.138866,6.99,1
142068,34634.0,0.333499,1.699873,-2.596561,3.643945,-0.585068,-0.654659,-2.275789,0.675229,-2.042416,...,0.469212,-0.144363,-0.317981,-0.769644,0.807855,0.228164,0.551002,0.305473,18.96,1
142069,96135.0,-1.952933,3.541385,-1.310561,5.955664,-1.003993,0.983049,-4.587235,-4.892184,-2.516752,...,-1.998091,1.133706,-0.041461,-0.215379,-0.865599,0.212545,0.532897,0.357892,18.96,1


BigQuery tables are organized into data blocks. The TABLESAMPLE clause works by randomly selecting a percentage of data blocks from the table and reading all of the rows in the selected blocks. The sampling granularity is limited by the number of data blocks.

Typically, BigQuery splits tables or table partitions into blocks if they are larger than about 1 GB. Smaller tables might consist of a single data block. In that case, the TABLESAMPLE clause reads the entire table. If the sampling percentage is greater than zero and the table is not empty, then table sampling always returns some results.

Blocks can be different sizes, so the exact fraction of rows that are sampled might vary. If you want to sample individual rows, rather than data blocks, then you can use a `WHERE rand() < K` clause instead. However, this approach requires BigQuery to scan the entire table. To save costs but still benefit from row-level sampling, you can combine both techniques.

In [7]:
# Construct a query to retrieve a sample of data from the BigQuery table
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` WHERE rand() < 0.1
#LIMIT 5
"""
# Execute the query and convert the result to a pandas dataframe for easy manipulation and review in Python
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,124543.0,1.981596,0.766670,-2.008529,3.545828,1.385911,-0.249485,0.885531,-0.183365,-1.732397,...,0.310266,0.849467,-0.102548,0.698778,0.613624,0.318235,-0.116135,-0.086042,0.00,0
1,133429.0,2.139338,-1.049756,-0.447567,-1.039456,-1.036828,-0.062044,-1.318024,0.166716,-0.048551,...,0.436919,1.242217,0.186753,0.723835,-0.249032,-0.112948,0.008805,-0.051290,0.00,0
2,150656.0,-2.752799,2.816138,2.150630,4.736698,-2.000659,2.301492,-2.702993,-0.410142,-0.366028,...,2.429036,-0.002044,0.143127,0.063521,0.431691,0.704297,0.381104,0.134873,0.00,0
3,29643.0,-0.908505,1.161986,1.928416,0.497961,-0.530008,-0.192915,0.096970,0.527777,-0.832962,...,-0.131507,-0.364673,-0.159579,0.574868,0.147281,0.284884,-0.079363,0.001679,0.00,0
4,51207.0,-0.454772,-0.154116,2.267192,-1.600022,-0.743330,-0.103501,-0.504217,0.226548,-0.618145,...,0.442227,1.310442,-0.212913,0.113222,-0.222111,-0.206106,0.164367,0.134539,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28252,97235.0,-17.537592,12.352519,-20.134613,11.122771,-14.571080,-0.381622,-23.928661,-4.724921,-8.603038,...,-4.352213,2.389041,2.019128,0.627192,-1.085997,-0.071803,-3.838198,-0.802564,9.82,1
28253,110552.0,-2.450367,2.107729,-5.140663,1.411304,-1.690780,-0.736427,-3.657946,1.944906,-0.788388,...,0.800538,0.364617,0.233608,-0.282078,-0.320311,0.492920,0.359976,-0.115471,80.22,1
28254,129186.0,0.290155,0.049243,-0.740524,2.865463,1.395294,-0.535163,0.142543,-0.222770,-1.463691,...,0.337349,1.018191,0.303550,0.833886,-1.222306,2.745261,-0.220402,0.168233,7.18,1
28255,29753.0,0.269614,3.549755,-5.810353,5.809370,1.538808,-2.269219,-0.824203,0.351070,-3.759059,...,0.371121,-0.322290,-0.549856,-0.520629,1.378210,0.564714,0.553255,0.402400,0.68,1


### [OPTIONAL] Check out this table in BigQuery Console:
Provides a direct link to the BigQuery console for further exploration and verification of the dataset.
This helps to manually inspect and validate the data directly in BigQuery's interface.

- Click: https://console.cloud.google.com/bigquery
- Make sure project selected is the one from this notebook
- Under Explore, expand this project and review the dataset and table

In [8]:
print(f"Direct Link To This Project In BigQuery:\nhttps://console.cloud.google.com/bigquery?project={PROJECT_ID}")

Direct Link To This Project In BigQuery:
https://console.cloud.google.com/bigquery?project=bootkon-test24mun-8303


### Review Data in BigQuery
Additional SQL queries could be used to review the data.  This section shows moving the table to a Pandas dataframe for local review in Python:
Reviewing the data helps in understanding its structure and quality before further processing.

> **Note:** <p>This query only selects one column.  This means BigQuery scans less data as it does not process the other columns.  </p>

In [9]:
# Construct a query to select a sample of the 'Class' column from the BigQuery table
query = f"""
SELECT Class
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""
# Execute the query and convert the result to a pandas dataframe for easy manipulation and review in Python

df = bq.query(query = query).to_dataframe()

In [10]:
# Display the value counts of the 'Class' column
# This helps in understanding the distribution of classes in the dataset.

df['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: Int64

In [11]:
# Display the normalized value counts of the 'Class' column
# Normalized value counts show the proportion of each class, which is useful for understanding class imbalance.

df['Class'].value_counts(normalize=True)

Class
0    0.998273
1    0.001727
Name: proportion, dtype: Float64

---
## Prepare Data for Analysis

Create a prepped version of the data with test/train splits using SQL DDL:

In [12]:
# Construct a BigQuery client object.
client = bigquery.Client()

# Get the credentials of the current environment, which should be the service account
credentials, project = default()
request = google.auth.transport.requests.Request()
credentials.refresh(request=request)

# print the service account
print(credentials.service_account_email) 
service_account_email = credentials.service_account_email

# Construct a query to create or replace a table with a train/test/validation split
# The CASE WHEN statement splits the data into training (80%), validation (10%), and test (10%) sets.

query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`) 
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits,
     "{service_account_email}" as service_account_email
FROM add_id
"""
# Execute the query to create the prepped table with the splits
job = bq.query(query = query)
job.result()

622854781065-compute@developer.gserviceaccount.com


In [13]:
# how many seconds the job took to be executed
(job.ended-job.started).total_seconds()

8.66

In [14]:
# Estimated bytes scan. To estimate the cost of the query in ON Demand pricing model
if job.estimated_bytes_processed:
    print(f'{job.estimated_bytes_processed/1000000} MB')

### Review the test/train split in BigQuery:

In [15]:
# Construct a query to review the distribution of splits
# This query counts the number of records in each split and calculates their percentage.

query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,VALIDATE,28570,10.031355
1,TEST,28466,9.994839
2,TRAIN,227771,79.973807


### Retrieve a subset of the data to a Pandas dataframe:
This allows for a quick inspection of the data in the notebook environment.

In [16]:
query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
LIMIT 5
"""

# Execute the query and convert the result to a pandas dataframe
data = bq.query(query = query).to_dataframe()

In [17]:
# Display the first few rows of the dataframe
# This provides a quick view of the data to verify the contents.

data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits,service_account_email
0,155682.0,2.078810,-0.054605,-1.410680,0.150717,0.020178,-1.344240,0.346070,-0.383459,0.706247,...,-0.008189,0.007838,-0.074078,-0.048061,-0.064491,0.0,0,ed0d6e96-3fc5-4297-b127-927594bffdf7,TEST,622854781065-compute@developer.gserviceaccount...
1,56949.0,1.116655,0.306354,1.370778,2.436085,-0.469814,0.569348,-0.570393,0.258074,-0.392261,...,0.020049,0.309446,-0.048418,0.038591,0.024479,0.0,0,009f1032-5a79-4af6-9c4d-b6daa330a055,TEST,622854781065-compute@developer.gserviceaccount...
2,72803.0,-2.026737,0.571666,1.447157,1.242799,-0.497292,0.493868,-1.083321,1.173734,-0.413958,...,-0.459467,0.308382,-0.059460,-0.329470,-0.194678,0.0,0,cd3561b6-f563-49a9-9a48-8e00a79d5747,TEST,622854781065-compute@developer.gserviceaccount...
3,122685.0,1.832669,0.132418,-0.129857,3.970067,-0.157261,0.490110,-0.422140,0.293274,-0.215628,...,-0.028606,-0.018597,0.121713,-0.015925,-0.061309,0.0,0,2b0346a7-81ee-4dbf-830d-8e1aa3089a75,TEST,622854781065-compute@developer.gserviceaccount...
4,23267.0,-0.408694,0.881913,2.285323,1.882568,0.649884,1.580303,0.000823,0.201674,1.282434,...,-1.436857,-0.159446,-0.044764,-0.225054,-0.173292,0.0,0,0011bece-bad7-491f-9db2-c0085b4a4b79,TEST,622854781065-compute@developer.gserviceaccount...
